# Importación de Librerías

In [ ]:
import sys
import os
sys.path.append(".")
import pandas as pd
import numpy as np
import pandasql as ps
from datetime import timedelta
from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
import seaborn as sns

color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')
pd.set_option('display.max_columns', None)
def thousands_formatter_func(x, pos):
    return f'{int(x / 1e3)}K'
thousand_formatter = FuncFormatter(thousands_formatter_func)
def percentage_formatter_func(x, pos):
    return f'{int(x * 100)}%'
percentage_formatter = FuncFormatter(percentage_formatter_func)

# Lectura de archivos

## Lectura Dataset de Bombas

In [ ]:
df_bombs = pd.read_csv('data/csv/Dataset_Bombes_2020-23.csv', delimiter=';', encoding='latin-1',  dtype={'T_Diff [s]': 'object'})
df_bombs['dhIni'] = pd.to_datetime(df_bombs['dhIni'], format='%Y-%m-%d %H:%M:%S.%f')
df_bombs['dhFin'] = pd.to_datetime(df_bombs['dhFin'], format='%Y-%m-%d %H:%M:%S.%f')
df_bombs['T_Diff [s]'] = df_bombs['T_Diff [s]'].fillna('0').str.replace('.', '').replace(',', '.').astype(int)
df_bombs['TiempoTeorico [min]'] = df_bombs['TiempoTeorico [min]'].fillna('0').str.replace('.', '').str.replace(',', '.').astype(float)
df_bombs['Orden'] = df_bombs['Orden'].fillna(0).astype(int)
df_bombs['OperIds'] = df_bombs['OperIds'].fillna('')

df_bombs.rename(columns={
    'Orden': 'order',
    'CodMaterial': 'bomb_type',
    'DescMaterial': 'bomb_description',
    'maquina_dsc': 'line',
    'QtyPlan': 'plan_qty',
    'QtyBuenas_Total': 'total_good_qty',
    'QtyMalas_Total': 'total_bad_qty',
    'CantidadBuenas': 'good_qty',
    'CantidadMalas': 'bad_qty',
    'dhIni': 'start_date',
    'dhFin': 'end_date',
    'T_Diff [s]': 'theorical_diff',
    'IncidName': 'incidence_name',
    'Observaciones': 'observations',
    'TiempoTeorico [min]': 'theorical_time',
    'OperariosEnMaq': 'operators_qty',
    'OperIds': 'operators_ids'    
}, inplace=True)

print(df_bombs.shape)
df_bombs.head()

## Lectura Dataset Defectuosas Interna

In [ ]:
df_defective = pd.read_csv('data/csv/Defectuositat_Interna_2020-23.csv', delimiter=';', encoding='latin-1')[:-3]
df_defective['Jornada'] = pd.to_datetime(df_defective['Jornada'], format='%d-%m-%y')
print(df_defective.shape)
print(df_defective.dtypes)
df_defective.head()


## Información Relevante de Bombas

In [ ]:
df_bombs.describe()

In [ ]:
df_exp = df_bombs.operators_ids.str.split(',', expand=True).stack().reset_index(level=1, drop=True).to_frame('operators_ids')

In [ ]:
print(f"""Resumen información relevante sobre el Dataset de bombas:
--> Cantidad de Órdenes de fabicación: {df_bombs.order.nunique()}
--> Cantidad de distintos materiales: {df_bombs.bomb_type.nunique()}
--> Cantidad de lineas/máquinas: {df_bombs.line.nunique()}
--> Cantidad máxima registrada de operarios en máquinas: {df_bombs.operators_qty.max()}
--> Cantidad de operarios distintos que trabajan en la planta: {df_exp.operators_ids.nunique()}

--> Fecha más antigua de registros: {df_bombs.start_date.min()}
--> Fecha más nueva de registros: {df_bombs.end_date.max()}    
""")

### Revisión de casos Diff cero

In [ ]:
print(df_bombs[df_bombs.theorical_diff < 0].order.unique())
df_bombs[df_bombs.theorical_diff < 0].order.nunique()

In [ ]:
df_bombs[df_bombs.order == 5263472]

In [ ]:
df_bombs[(df_bombs.theorical_diff < 0) & (df_bombs.good_qty != 0)].order.nunique()

Conclusión: eliminar la OF que contengan registros?

## Generación cantidad teórica calculada por registro

Creamos nuestra propias columnas calculadas: <br>
* Cantidad de trabajadores por registro según los ids asociados.
* Diferencia en segundos de tiempo de inicio con tiempo de fin de registro
* Diferencia en minutos de tiempo de inicio con tiempo de fin de registro
* Total de *minutos-hombre* trabajados en el registro ${duración de registo en minutos \times cantidad de operarios en el registro}$

In [ ]:
df_bombs['operators_qty_calculated'] = df_bombs.operators_ids.apply(lambda x: len(list(filter(lambda s: s != "", x.split(',')))))
df_bombs['time_diff_seconds_calculated'] = (df_bombs['end_date'] - df_bombs['start_date']) / pd.Timedelta(seconds=1)
df_bombs['time_diff_minutes_calculated'] = (df_bombs['end_date'] - df_bombs['start_date']) / pd.Timedelta(minutes=1)

df_bombs['total_operators_minutes'] = df_bombs['operators_qty_calculated'] * df_bombs['time_diff_minutes_calculated']

df_bombs

## Agrupación por Orden de fabricación

In [ ]:
df_bombs_grouped = df_bombs.groupby(
        ['order', 'bomb_type', 'line']
    ).agg(
        plan_qty=('plan_qty', 'max'),
        total_good_qty=('total_good_qty', 'max'),
        total_bad_qty=('total_bad_qty', 'max'),
        good_qty=('good_qty', 'sum'),
        bad_qty=('bad_qty', 'sum'),
        start_date=('start_date', 'min'),
        end_date=('end_date', 'max'),
        theorical_diff=('theorical_diff', 'sum'),
        theorical_time=('theorical_time', 'max'),
        time_diff_seconds_calculated=('time_diff_seconds_calculated', 'sum'),
        time_diff_minutes_calculated=('time_diff_minutes_calculated', 'sum'),
        total_operators_minutes=('total_operators_minutes', 'sum'),
        operators_distinct_ids=('operators_ids', lambda x: list(set([i for s in x if isinstance(s, str) for i in s.split(',') if i]))),
        registers_qty=('start_date', 'count'),
        operators_each_registers_qty=('operators_qty_calculated', 'mean'),
    ).reset_index()
df_bombs_grouped['operators_distinct_qty'] = df_bombs_grouped.operators_distinct_ids.apply(lambda x: len(x))
df_bombs_grouped['theorical_qty'] = df_bombs_grouped.total_operators_minutes/df_bombs_grouped.theorical_time
df_bombs_grouped['theorical_qty_round'] = df_bombs_grouped.theorical_qty.round(0)
df_bombs_grouped['performance'] = df_bombs_grouped.good_qty/df_bombs_grouped.theorical_qty
df_bombs_grouped['performance_round'] = df_bombs_grouped.good_qty/df_bombs_grouped.theorical_qty_round
df_bombs_grouped

In [ ]:
df_bombs_grouped.describe()

In [ ]:
df_bombs_grouped.operators_each_registers_qty = df_bombs_grouped.operators_each_registers_qty.round(0)
df_bombs_grouped['total_theorical_time'] = df_bombs_grouped.operators_each_registers_qty*df_bombs_grouped.theorical_time
df_bombs_grouped

In [ ]:
data = df_bombs_grouped[df_bombs_grouped.good_qty < 3000]
plt.scatter(data.operators_each_registers_qty, data.good_qty, alpha=1)
plt.title('Relación entre cantidad de operadores a trabajar y cantidad a elaborar en la OF')
plt.xlabel('Media operador por registro en cada OF')
plt.ylabel('Cantidad a elaborar de OF')
plt.show()

In [ ]:
data = df_bombs_grouped[df_bombs_grouped.good_qty < 3000]
plt.scatter(data.operators_each_registers_qty, data.total_theorical_time, alpha=1)
plt.title('Relación entre cantidad de operadores a trabajar y cantidad a elaborar en la OF')
plt.xlabel('Media operador por registro en cada OF')
plt.ylabel('Cantidad a elaborar de OF')
plt.show()

In [ ]:
df_of = df_bombs_grouped.copy()

Más información relevante:

In [ ]:
print(f'''
    | Cantidad total órdenes de fabricación: {df_of.shape[0]}
    | Cantidad de órdenes de fabricación con diferencia en cantidad buenas reportadas versus la suma de sus registros: {df_of[df_of.total_good_qty != df_of.good_qty].shape[0]}
        -> Casos CantidadBuenas > QtyBuenas_total : {df_of[df_of.total_good_qty < df_of.good_qty].shape[0]}  
        -> Casos CantidadBuenas < QtyBuenas_total : {df_of[df_of.total_good_qty > df_of.good_qty].shape[0]} 
    
    | Cantidad máxima de operadores trabajando en una misma OF: {df_of.operators_distinct_qty.max()}
    
    | Cantidad de órdenes con Qtyplan distintos a cantidad teórica calculada: {df_of[df_of.plan_qty != df_of.theorical_qty_round].shape[0]}
    
    | Duración máxima (minutos) de un registro: {df_bombs.time_diff_minutes_calculated.max()}
''')

Revisión de cantidad de OF operadas por línea:

In [ ]:
df_of.groupby('line').size().reset_index(name='count').sort_values(by='count', ascending=False)

Revisión de cantidad de tipos de bombas operadas por máquina:

In [ ]:
df_bombs.bomb_type.nunique()

In [ ]:
df_of.groupby(['line', 'bomb_type']).size().reset_index().groupby(['line']).size().reset_index(name='count').sort_values(by='count', ascending=False)

Revisión de líneas por bomba:

In [ ]:
df_grouped = df_of.groupby(['line', 'bomb_type']).size().reset_index().groupby(['bomb_type']).size().reset_index(name='lines_count').sort_values(by='lines_count', ascending=False)
print(df_grouped[df_grouped.lines_count >= 2].shape[0])
df_grouped[df_grouped.lines_count >= 2]

Revisión de tipos de bomba por orden de fabricación

In [ ]:
df_grouped = df_of.groupby(['order', 'bomb_type']).size().reset_index().groupby(['order']).size().reset_index(name='count').sort_values(by='count', ascending=False)
df_grouped[df_grouped['count'] > 1]

>Nota: órdenes de fabricación hace alusión a un único tipo de bomba

## Limpieza

Eliminación de OF con tiempos negativos (confirmar con cliente)

In [ ]:
negative_of = df_bombs[df_bombs.time_diff_minutes_calculated < 0].order.unique()
negative_of.shape[0]


In [ ]:
df_of = df_of[~df_of.order.isin(negative_of)]
df_bombs = df_bombs[~df_bombs.order.isin(negative_of)]

Revisión de performance de OFs

In [ ]:
print(f'''Total OFs: {df_of.shape[0]}
OFs con performance mayor a 100%: {df_of[df_of.performance > 1.0].shape[0]} | {round(df_of[df_of.performance > 1.0].shape[0]/df_of.shape[0]*100)}% del total
OFs con performance mayor a 110%: {df_of[df_of.performance > 1.1].shape[0]} | {round(df_of[df_of.performance > 1.1].shape[0]/df_of.shape[0]*100)}% del total
OFs con performance mayor a 120%: {df_of[df_of.performance > 1.2].shape[0]} | {round(df_of[df_of.performance > 1.2].shape[0]/df_of.shape[0]*100)}% del total
OFs con performance mayor a 150%: {df_of[df_of.performance > 1.5].shape[0]} | {round(df_of[df_of.performance > 1.5].shape[0]/df_of.shape[0]*100)}% del total
''')

Eliminación de OF con performance > 120% (confirmar con cliente)

In [ ]:
over_performance_of = df_of[df_of.performance > 1.2].order.unique()
over_performance_of.shape[0]

In [ ]:
df_of = df_of[~df_of.order.isin(over_performance_of)]
df_bombs = df_bombs[~df_bombs.order.isin(over_performance_of)]

Eliminación de órdenes de fabricación en distintas líneas

In [ ]:
df_grouped = df_of.groupby(['order', 'line']).size().reset_index().groupby(['order']).size().reset_index(name='count').sort_values(by='count', ascending=False)
multiple_lines_of = df_grouped[df_grouped['count'] > 1].order.unique()
multiple_lines_of.shape[0]

In [ ]:
df_of = df_of[~df_of.order.isin(multiple_lines_of)]
df_bombs = df_bombs[~df_bombs.order.isin(multiple_lines_of)]

# EDA

## Exploración de Dataset de bombas

### Exploración de valores por columnas

In [ ]:
df_bombs.head()

Revisión de cantidad de valores distintos por columna

In [ ]:
results = []
for col in df_bombs.columns:
    results.append([col, len(df_bombs[col].unique())])
pd.DataFrame(results, columns=['Column', 'Distinct Values'])

Revisión de tipos de incidencias registrados:

In [ ]:
df_bombs.groupby(['incidence_name']).size().reset_index(name='count')

> Nota: existen registros que tiene incidencia nula. Por lo observado esto se debe a que la observación de ciertos registros abarca más de una celda en el excel. A validar con el cliente qué hacer con estos casos

### Exploración de desempeño general por máquina

In [ ]:
df_bombs.groupby(['line']).agg(good_qty=('good_qty', 'sum'), bad_qty=('bad_qty', 'sum'))


In [ ]:
df_bombs_grouped = df_of.groupby(['line']).agg(theorical_qty=('theorical_qty', 'sum'), theorical_qty_round=('theorical_qty_round', 'sum'), good_qty=('good_qty', 'sum'))
df_bombs_grouped['success_percentage'] = (df_bombs_grouped.good_qty / df_bombs_grouped.theorical_qty *100).round(2) 
df_bombs_grouped['success_percentage_round'] = (df_bombs_grouped.good_qty / df_bombs_grouped.theorical_qty_round *100).round(2)
df_bombs_grouped

### Gráficas de interés


#### ¿Cuál es el rango de fechas en el dataset? ¿Existen periodos de inactividad notables en la producción?

In [ ]:
def plot_daily_production(df, title):
    data_daily = df.resample('D', on='start_date').sum()
    plt.figure(figsize=(10, 5))
    plt.plot(data_daily.index, data_daily['good_qty'])
    plt.xlabel('Fecha')
    plt.ylabel('Cantidad de producción')
    plt.title(title)
    plt.show()
    

>Gráfico 1: Series temporales de producción a lo largo del tiempo (por día)

In [ ]:
plot_daily_production(df_bombs, 'Producción diaria empresa a lo largo tiempo')

>Gráfico 2: Series temporales de producción a lo largo del tiempo (por día) para el año 2020

In [ ]:
year = 2020
plot_daily_production(df_bombs[df_bombs.start_date.dt.year == year], f'Producción diaria empresa en el año {year}')

>Gráfico 3: Comparación producción mensual por año

In [ ]:
data = df_bombs.copy()
# Crear una columna solo con el mes y el día
data['month'] = data.end_date.dt.strftime('%m')

# Agrupar por año, mes y día, y calcular la cantidad de producción diaria
data_daily = data.groupby([data.end_date.dt.year, 'month'])['good_qty'].sum().reset_index()
data_daily.columns = ['year', 'month', 'good_qty']

# Función para formatear las etiquetas del eje y
def thousands_formatter(x, pos):
    return f'{int(x / 1e3)}K'

# Crear el gráfico
plt.figure(figsize=(12, 6))

# Obtener los años únicos
unique_years = data_daily['year'].unique()

# Colors
color_pal = sns.color_palette(['steelblue', 'seagreen', 'gray', 'gold', 'magenta', 'yellow'])

for i, year in enumerate(unique_years):
    yearly_data = data_daily[data_daily['year'] == year]
    plt.plot(yearly_data['month'], yearly_data['good_qty'], label=str(int(year)), color=color_pal[i])

plt.xlabel('Fecha (Mes)')
plt.ylabel('Cantidad de producción')
plt.title('Producción mensual por año')

# Aplicar el formateador al eje y
formatter = FuncFormatter(thousands_formatter)
plt.gca().yaxis.set_major_formatter(formatter)

plt.legend(title='Año')
plt.xticks(rotation=45, ha='right')
plt.show()

#### ¿Cuáles son los tipos de bombas más y menos producidos?

In [ ]:
df_bombs.bomb_type.nunique()

Dada la cantidad de tipos de bombas, revisaremos las 20 más producidos y las 20 menos producidos

>Gráfico 4: Cantidad producida de las 20 tipos de bombas más demandados

In [ ]:
material_counts = df_bombs.groupby(['bomb_type', 'bomb_description'])['good_qty'].size().reset_index(name='count').sort_values(by='count', ascending=False)
material_counts['code_description'] = '[' + material_counts['bomb_type'].astype(str) + '] - ' + material_counts['bomb_description']
material_counts[0:20].plot(kind='bar', x='code_description', figsize=(10, 5))
plt.xlabel('Tipo de bomba')
plt.ylabel('Cantidad producida')
plt.title('Cantidad producida de los 20 bombas más demandadas')
plt.xticks(rotation=70, ha='right', fontsize=8)
plt.gca().legend().set_visible(False)
plt.show()

>Gráfico 4: Cantidad producida de los 20 materiales menos demandados

In [ ]:
material_counts[-20:].plot(kind='bar', x='code_description', figsize=(10, 5))
plt.xlabel('Material')
plt.ylabel('Cantidad producida')
plt.title('Cantidad producida de los 20 materiales más demandados')
plt.xticks(rotation=70, ha='right', fontsize=8)
plt.gca().legend().set_visible(False)
plt.show()

#### ¿Cuáles son los materiales más defectuosos?

Dada la cantidad de materiales, revisaremos los 20 más defectuosos y los 20 menos defectuosos
>Se revisa en la data que entre todos los registros, no hay ninguno que tenga QtyMalas_Total y CantidadMalas

#### ¿Cuáles son las máquinas que más y menos producen?

In [ ]:
bomb_counts = df_bombs.groupby(['line'])['good_qty'].sum().reset_index(name='count').sort_values(by='count', ascending=False)
bomb_counts.plot(kind='bar', x='line', figsize=(10, 5))
plt.xlabel('Línea')
plt.ylabel('Cantidad producida [bombas]')
plt.title('Cantidad de bombas producidas por línea')
plt.xticks(rotation=70, ha='right', fontsize=8)
plt.gca().yaxis.set_major_formatter(thousand_formatter)
plt.gca().legend().set_visible(False)
plt.show()

In [ ]:
bomb_counts = df_bombs.groupby(['line'])['order'].size().reset_index(name='count').sort_values(by='count', ascending=False)
bomb_counts.plot(kind='bar', x='line', figsize=(10, 5))
plt.xlabel('Máquina')
plt.ylabel('Cantidad producida [OFs]')
plt.title('Cantidad de OFs trabajadas por linea')

plt.xticks(rotation=70, ha='right', fontsize=8)
plt.gca().yaxis.set_major_formatter(thousand_formatter)
plt.gca().legend().set_visible(False)
plt.show()

#### ¿Cuáles son los tiempos de producción de las lineas?

In [ ]:
df_of[df_of.time_diff_minutes_calculated > 6000]

In [ ]:
df_of['time_diff_hours_calculated'] = df_of.time_diff_minutes_calculated/60

In [ ]:
plt.figure(figsize=(10, 10))
sns.boxplot(data=df_of[df_of.time_diff_minutes_calculated < 6000], x='line', y='time_diff_minutes_calculated')
plt.xlabel('Línea')
plt.ylabel('Tiempo de producción (minutos)')
plt.title('Tiempo de producción por OF por linea')
plt.xticks(rotation=70, ha='right', fontsize=8)
plt.show()

In [ ]:
plt.figure(figsize=(10, 10))
sns.boxplot(data=df_of[df_of.time_diff_minutes_calculated < 6000], x='line', y='time_diff_hours_calculated')
plt.xlabel('Línea')
plt.ylabel('Tiempo de producción (minutos)')
plt.title('Tiempo de producción por OF por linea')
plt.xticks(rotation=70, ha='right', fontsize=8)
plt.show()

#### Gráficas de performance por linea

In [ ]:
plt.figure(figsize=(10, 10))
sns.boxplot(data=df_of, x='line', y='performance')
plt.xlabel('Línea')
plt.ylabel('Performance (%)')
plt.title('Performance OFs por línea')
plt.xticks(rotation=70, ha='right', fontsize=8)
plt.show()

#### ¿Cuáles es la relación entre la cantidad teórica y la cantidad real?

In [ ]:
lines = df_of.line.unique()

for line in lines:
    data = df_of[df_of.line == line].sort_values(by='theorical_qty')

    plt.figure(figsize=(10, 5))
    sns.histplot(data=data, x='theorical_qty', bins=20, kde=True, label='Teórico')
    sns.histplot(data=data, x='good_qty', bins=20, kde=True, label='Real')
    plt.xlabel('Cantidad de producción teórica versus real')
    plt.ylabel('Frecuencia')
    plt.title(f'Distribución de la cantidad teórica de producción de la {line}')
    plt.xticks(rotation=70, ha='right', fontsize=8)
    plt.gca().legend().set_visible(True)
    plt.show()

#### Performance

In [ ]:
data = df_of.sort_values(by='performance')

plt.figure(figsize=(10, 5))
sns.histplot(data=data, x='performance', bins=100, kde=True)
plt.xlabel('Performance por OF')
plt.ylabel('Frecuencia')
plt.title('Distribución de la performance en todas las OFs de la empresa')
plt.xticks(rotation=70, ha='right', fontsize=8)
plt.gca().xaxis.set_major_formatter(percentage_formatter)
plt.show()

In [ ]:
lines = df_of.line.unique()

for line in lines:
    data = df_of[df_of.line == line].sort_values(by='performance')

    plt.figure(figsize=(10, 5))
    sns.histplot(data=data, x='performance', bins=100, kde=True)
    plt.xlabel('Performance de produccicón teórica versus real')
    plt.ylabel('Frecuencia')
    plt.title(f'Distribución de la performance de producción de la {line}')
    plt.xticks(rotation=70, ha='right', fontsize=8)
    plt.gca().xaxis.set_major_formatter(percentage_formatter)
    plt.show()

In [ ]:
df_gouped = df_of.groupby('line').agg(of_qty=('order', 'count'), mean_performance=('performance', 'mean')).reset_index().sort_values(by='mean_performance')
df_gouped

In [ ]:
df_gouped.plot(kind='bar', x='line', y='mean_performance', figsize=(10, 5))
plt.xlabel('Línea')
plt.ylabel('Media Performance')
plt.title('Media de Performance por linea')

plt.xticks(rotation=70, ha='right', fontsize=8)
plt.gca().yaxis.set_major_formatter(percentage_formatter)
plt.gca().legend().set_visible(False)
plt.show()

## Exploración por operarios

### Generación de dataframe operadores-OFs en general

In [ ]:
df_of.head()

In [ ]:
data_operators = {
    'id': [], 'of_ids': [], 'of_performances': [], 'of_qty': [],
    'line_ids': [], 'lines_qty': [], 'bomb_type_ids': [], 'bomb_types_qty': [],
    'min_registered_date': [], 'max_registered_date': []
}

for index, row in df_of.iterrows():
    for operator_id in row['operators_distinct_ids']:
        if operator_id not in data_operators['id']:
            data_operators['id'].append(operator_id)
            data_operators['of_ids'].append([row['order']]) 
            data_operators['of_performances'].append([row['performance']])
            data_operators['of_qty'].append(1)
            data_operators['line_ids'].append([row['line']])
            data_operators['lines_qty'].append(1)
            data_operators['bomb_type_ids'].append([row['bomb_type']])
            data_operators['bomb_types_qty'].append(1)
            data_operators['min_registered_date'].append(row['start_date'])
            data_operators['max_registered_date'].append(row['end_date'])
        else:
            oper_index = data_operators['id'].index(operator_id)
            data_operators['of_ids'][oper_index].append(row['order'])
            data_operators['of_performances'][oper_index].append(row['performance'])
            data_operators['of_qty'][oper_index] += 1
            if row['line'] not in data_operators['line_ids'][oper_index]:
                data_operators['line_ids'][oper_index].append(row['line'])
                data_operators['lines_qty'][oper_index] += 1
            if row['bomb_type'] not in data_operators['bomb_type_ids'][oper_index]:
                data_operators['bomb_type_ids'][oper_index].append(row['bomb_type'])
                data_operators['bomb_types_qty'][oper_index] += 1
            if row['start_date'] < data_operators['min_registered_date'][oper_index]:
                data_operators['min_registered_date'][oper_index] = row['start_date']
            if row['end_date'] > data_operators['max_registered_date'][oper_index]:
                data_operators['max_registered_date'][oper_index] = row['end_date']
     
     
df_operators = pd.DataFrame(data_operators)            
df_operators['mean_performance'] = df_operators.of_performances.apply(lambda x: sum(x)/len(x))
df_operators['std_performance'] = df_operators.of_performances.apply(lambda x: np.std(x))
df_operators['min_performance'] = df_operators.of_performances.apply(lambda x: min(x))
df_operators['max_performance'] = df_operators.of_performances.apply(lambda x: max(x))
print(df_operators.shape)
df_operators.head()

Información de cómo distribuyen los operadores

In [ ]:
print(f'''
Cantidad de trabajadores: {df_operators.id.nunique()}
Lineas por trabajador --> Media: {round(df_operators.lines_qty.mean(), 3)} | Min: {df_operators.lines_qty.min()} | Max: {df_operators.lines_qty.max()}
Bombas por trabajador --> Media: {round(df_operators.bomb_types_qty.mean(), 3)} | Min: {df_operators.bomb_types_qty.min()} | Max: {df_operators.bomb_types_qty.max()}
''')

In [ ]:
data = df_operators.sort_values(by='mean_performance')

plt.figure(figsize=(10, 5))
sns.histplot(data=data, x='mean_performance', bins=9, kde=True)
plt.xlabel('Media de performance de operadores')
plt.ylabel('Frecuencia')
plt.title('Distribución de la media de performance de operadores')
plt.xticks(rotation=70, ha='right', fontsize=8)
plt.gca().xaxis.set_major_formatter(percentage_formatter)

plt.show()

Performance promedio operadores

In [ ]:
sns.set_style('white')
data = df_operators.sort_values(by='lines_qty')

plt.figure(figsize=(10, 5))
sns.histplot(data=data, x='lines_qty', bins=9, kde=True)

plt.xlabel('Cantidad de lineas distintas en que se ha trabajado')
plt.ylabel('Frecuencia')
plt.title('Distribución del trabajo de operadores en líneas distintas')
plt.xticks(rotation=70, ha='right', fontsize=8)

plt.show()

In [ ]:
sns.set_style('white')
data = df_operators.sort_values(by='bomb_types_qty')

plt.figure(figsize=(10, 5))
sns.histplot(data=data, x='bomb_types_qty', bins=30, kde=True)

plt.xlabel('Cantidad de tipos de bombas distintas en que se ha trabajado')
plt.ylabel('Frecuencia')
plt.title('Distribución del trabajo de operadores en tipos de bombas distintas')
plt.xticks(rotation=70, ha='right', fontsize=8)

plt.show()

### Generación de dataframe operadores-OF-participación

In [ ]:
operator_data = {}

for index, row in df_bombs.iterrows():
    order = row['order']
    operators = [op for op in row['operators_ids'].split(',') if op] 
    time_diff_minutes = row['time_diff_minutes_calculated']

    for operator in operators:
        if operator and (operator, order) not in operator_data:
            operator_data[(operator, order)] = 0
        operator_data[(operator, order)] += time_diff_minutes

new_df_data = []
for (operator, order), time in operator_data.items():
    new_df_data.append({'operator_id': operator, 'order': order, 'participation_minutes': time})

df_operators_participation = pd.DataFrame(new_df_data)
df_operators_participation.head()

In [ ]:
df_of_index = df_of.set_index('order')

def get_info(row, col):
    return df_of_index.loc[row['order'], col]

def calculate_percentage(row):
    order = row['order']
    participation_minutes = row['participation_minutes']
    total_operators_minutes = df_of_index.loc[order, 'total_operators_minutes']
    return round((participation_minutes / total_operators_minutes) * 100, 1)

df_operators_participation['total_operators_minutes'] = df_operators_participation.apply(lambda row: get_info(row, 'total_operators_minutes'), axis=1)
df_operators_participation['participation_percentage'] = df_operators_participation.apply(calculate_percentage, axis=1)
df_operators_participation['of_performance'] = df_operators_participation.apply(lambda row: get_info(row, 'performance'), axis=1)
df_operators_participation['bomb_type'] = df_operators_participation.apply(lambda row: get_info(row, 'bomb_type'), axis=1)
df_operators_participation['good_qty'] = df_operators_participation.apply(lambda row: get_info(row, 'good_qty'), axis=1)
df_operators_participation['line'] = df_operators_participation.apply(lambda row: get_info(row, 'line'), axis=1)
df_operators_participation['theorical_qty'] = df_operators_participation.apply(lambda row: get_info(row, 'theorical_qty'), axis=1)
df_operators_participation['production_date'] = df_operators_participation.apply(lambda row: get_info(row, 'end_date'), axis=1)

df_operators_participation

In [ ]:
df_operators_participation.groupby(['operator_id', 'line']).agg(orders_qty=('order', 'count'),).reset_index().sort_values(by='orders_qty', ascending=False)

In [ ]:
df_grouped = df_operators_participation.groupby(['operator_id', 'line']).agg(orders_qty=('order', 'count'),).reset_index().groupby(['operator_id']).agg(lines_qty=('line', 'count'),).reset_index()
df_grouped[df_grouped.lines_qty > 1]

In [ ]:
data = df_grouped.sort_values(by='lines_qty')

plt.figure(figsize=(10, 5))
sns.histplot(data=data, x='lines_qty', bins=8, kde=True)
plt.xlabel('Cantidad de líneas distintas')
plt.ylabel('Cantidad trabajadores')
plt.title('Cantidad de líneas distintas en que los operadores trabajan')
plt.xticks(rotation=70, ha='right', fontsize=8)

plt.show()

Cantidad de operadores según distintas líneas en las que han trabajado


In [ ]:
df_grouped.groupby(['lines_qty']).agg(operators_qty=('operator_id', 'count'),).reset_index()

## Exploración patrones en performance

### ¿Cómo es la performance de los trabajadores de la fábrica?

In [ ]:
df_operators.head()

#### Definición de segmentos

In [ ]:
df_op_1 = df_operators[(df_operators.mean_performance > 1.0)]
df_op_2 = df_operators[(df_operators.mean_performance > .9) & (df_operators.mean_performance <= 1.0)]
df_op_3 = df_operators[(df_operators.mean_performance > .8) & (df_operators.mean_performance <= .9)]
df_op_4 = df_operators[(df_operators.mean_performance > .7) & (df_operators.mean_performance <= .8)]
df_op_5 = df_operators[(df_operators.mean_performance > .6) & (df_operators.mean_performance <= .7)]
df_op_6 = df_operators[(df_operators.mean_performance > .5) & (df_operators.mean_performance <= .6)]
df_op_7 = df_operators[(df_operators.mean_performance <= .5)]

In [ ]:
operators_seg_dict = {
'[> 100%]': df_op_1,
'[90% - 100%]': df_op_2,
'[80% - 90%]': df_op_3,
'[70% - 80%]': df_op_4,
'[60% - 70%]': df_op_5,  
'[50% - 60%]': df_op_6,
'[< 50%]': df_op_7    
}

In [ ]:
print(f'''
  Rendimientos promedios de operadores:
  
  [> 100%] --> {df_op_1.shape[0]} | {round(df_op_1.shape[0]/df_operators.shape[0]*100, 2)}% 
  [90% - 100%] --> {df_op_2.shape[0]} | {round(df_op_2.shape[0]/df_operators.shape[0]*100, 2)}% 
  [80% - 90%] --> {df_op_3.shape[0]} | {round(df_op_3.shape[0]/df_operators.shape[0]*100, 2)}% 
  [70% - 80%] --> {df_op_4.shape[0]} | {round(df_op_4.shape[0]/df_operators.shape[0]*100, 2)}% 
  [60% - 70%] --> {df_op_5.shape[0]} | {round(df_op_5.shape[0]/df_operators.shape[0]*100, 2)}% 
  [50% - 60%] --> {df_op_6.shape[0]} | {round(df_op_6.shape[0]/df_operators.shape[0]*100, 2)}% 
  [< 50%] --> {df_op_7.shape[0]} | {round(df_op_7.shape[0]/df_operators.shape[0]*100, 2)}% 
''')

In [ ]:
labels = ['> 100%', '90%-100%', '80%-90%', '70%-80%', '60%-70%', '50%-60%', '< 50%']
values = [3, 17, 48, 26, 8, 7, 5]
percentages = [2.63, 14.91, 42.11, 22.81, 7.02, 6.14, 4.39]

# Crear el gráfico de barras
fig, ax = plt.subplots()
bars = ax.bar(labels, values)

# Agregar porcentajes sobre las barras
for i, bar in enumerate(bars):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width() / 2, height + 0.5, f'{percentages[i]:.2f}%', ha='center', va='bottom')

# Configurar etiquetas y título
ax.set_ylabel('Cantidad de operadores')
ax.set_title('Cantidad de operadores según\nsegmento de performance promedio', pad=30)
plt.xticks(rotation=75)

# Mostrar el gráfico
plt.show()

### Evaluación de cada segmento

In [ ]:
print(f'''
Promedio de porcentaje de participación (sobre total de la OF) y cantidad promedio de OFs trabajadas según cada segmento:
[> 100%] --> {round(df_operators_participation[df_operators_participation.operator_id.isin(df_op_1.id.unique())].participation_percentage.mean(), 2)}% | {round(df_op_1.of_qty.mean(), 1)}
[90% - 100%] --> {round(df_operators_participation[df_operators_participation.operator_id.isin(df_op_2.id.unique())].participation_percentage.mean(), 2)}% | {round(df_op_2.of_qty.mean(), 1)} 
[80% - 90%] --> {round(df_operators_participation[df_operators_participation.operator_id.isin(df_op_3.id.unique())].participation_percentage.mean(), 2)}% | {round(df_op_3.of_qty.mean(), 1)} 
[70% - 80%] --> {round(df_operators_participation[df_operators_participation.operator_id.isin(df_op_4.id.unique())].participation_percentage.mean(), 2)}% | {round(df_op_4.of_qty.mean(), 1)}  
[60% - 70%] --> {round(df_operators_participation[df_operators_participation.operator_id.isin(df_op_5.id.unique())].participation_percentage.mean(), 2)}% | {round(df_op_5.of_qty.mean(), 1)} 
[50% - 60%] --> {round(df_operators_participation[df_operators_participation.operator_id.isin(df_op_6.id.unique())].participation_percentage.mean(), 2)}% | {round(df_op_6.of_qty.mean(), 1)}  
[< 50%] --> {round(df_operators_participation[df_operators_participation.operator_id.isin(df_op_7.id.unique())].participation_percentage.mean(), 2)}% | {round(df_op_7.of_qty.mean(), 1)}       
''')

In [ ]:
data = df_of[df_of.total_operators_minutes < 12000] # quitar outlier de 17392 minutos-hombre
plt.scatter(data['total_operators_minutes'], data['performance'], alpha=0.5)
plt.title('Relación entre minutos-hombre y performance')
plt.xlabel('Minutos-hombre')
plt.ylabel('Performance')
plt.show()

#### Participación por línea según cada segmento

In [ ]:
for seg, df_op in operators_seg_dict.items():
    data = df_operators_participation[df_operators_participation.operator_id.isin(df_op.id.unique())]
    data = data.groupby(['line'])['order'].size().reset_index(name='count').sort_values(by='line', ascending=False)
    plt.bar(data['line'], data['count'], label=str(seg))

plt.xlabel('Línea')
plt.ylabel('Participación operarios')
plt.title('Participación de operarios por línea por segmento')

# Aplicar el formateador al eje y
formatter = FuncFormatter(thousands_formatter)
plt.gca().yaxis.set_major_formatter(formatter)

plt.legend(title='Segmento')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
segments_list= ['[> 100%]', '[90% - 100%]']
for seg, df_op in {key: value for key, value in operators_seg_dict.items() if key in segments_list}.items():
    data = df_operators_participation[df_operators_participation.operator_id.isin(df_op.id.unique())]
    data = data.groupby(['line'])['order'].size().reset_index(name='count').sort_values(by='line', ascending=False)
    plt.bar(data['line'], data['count'], label=str(seg))

plt.xlabel('Línea')
plt.ylabel('Participación operarios')
plt.title(f'Participación de operarios por línea\npara segmentos {segments_list}')

plt.legend(title='Segmento')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
segments_list= ['[80% - 90%]', '[70% - 80%]', '[60% - 70%]', '[50% - 60%]', '[< 50%]']
for seg, df_op in {key: value for key, value in operators_seg_dict.items() if key in segments_list}.items():
    data = df_operators_participation[df_operators_participation.operator_id.isin(df_op.id.unique())]
    data = data.groupby(['line'])['order'].size().reset_index(name='count').sort_values(by='line', ascending=False)
    plt.bar(data['line'], data['count'], label=str(seg))

plt.xlabel('Línea')
plt.ylabel('Participación operarios')
plt.title('Participación de operarios por línea\n por segmento de performance')

plt.legend(title='Segmento')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
segments_list= ['[> 100%]', '[< 50%]']
for seg, df_op in {key: value for key, value in operators_seg_dict.items() if key in segments_list}.items():
    data = df_operators_participation[df_operators_participation.operator_id.isin(df_op.id.unique())]
    data = data.groupby(['line'])['order'].size().reset_index(name='count').sort_values(by='line', ascending=False)
    plt.bar(data['line'], data['count'], label=str(seg))

plt.xlabel('Línea')
plt.ylabel('Participación operarios')
plt.title('Participación de operarios por línea\n por segmento de performance')

plt.legend(title='Segmento')
plt.xticks(rotation=45, ha='right')
plt.show()

##### Detalle por segmento

In [ ]:
for seg, df_op in operators_seg_dict.items():
    data = df_operators_participation[df_operators_participation.operator_id.isin(df_op.id.unique())]
    data = data.groupby(['line'])['order'].size().reset_index(name='count').sort_values(by='line', ascending=False)
    data.plot(kind='bar', x='line', figsize=(10, 5))
    plt.xlabel('Línea')
    plt.ylabel('Frecuencia participaciones operadores')
    plt.title(f'Cantidad participaciones por línea en segmento {seg}')
    plt.xticks(rotation=70, ha='right', fontsize=8)
    plt.gca().legend().set_visible(False)
    plt.show()



#### Participación por rangos sobre el tiempo de participación en el tiempo total de las OFs

In [ ]:
data = df_operators_participation
data['participation_category'] = pd.cut(data['participation_percentage'], 
                                        bins=range(0, 105, 5),
                                        labels=[f'{i}% - {i+5}%' for i in range(0, 100, 5)]).fillna('0% - 5%')
data = data.groupby(['participation_category'])['order'].size().reset_index(name='count').sort_values(by='participation_category', ascending=True)
plt.bar(data['participation_category'], data['count'])

plt.xlabel('Porcentaje de participación sobre tiempo total OF')
plt.ylabel('Frecuencia')
plt.title('Frecuencia % de participación en tiempo total')

plt.xticks(rotation=90, fontsize=8, ha='center')
plt.show()

In [ ]:
segments_list= ['[80% - 90%]', '[70% - 80%]', '[60% - 70%]', '[50% - 60%]', '[< 50%]']
segments_list= operators_seg_dict.keys()
data = df_operators_participation
data['participation_category'] = pd.cut(data['participation_percentage'], 
                                        bins=range(0, 105, 5),
                                        labels=[f'{i}% - {i+5}%' for i in range(0, 100, 5)]).fillna('0% - 5%')
for seg, df_op in {key: value for key, value in operators_seg_dict.items() if key in segments_list}.items():
    data_aux = df_operators_participation[df_operators_participation.operator_id.isin(df_op.id.unique())]
    data_aux = data_aux.groupby(['participation_category'])['order'].size().reset_index(name='count').sort_values(by='participation_category', ascending=True)
    plt.bar(data_aux['participation_category'], data_aux['count'], label=str(seg))

plt.xlabel('Porcentaje de participación sobre tiempo total OF')
plt.ylabel('Participación operarios')
plt.title('Participación de operarios por línea por segmento de performance')

plt.legend(title='Segmento')
plt.xticks(rotation=90, fontsize=8, ha='center')
plt.show()

In [ ]:
segments_list= ['[< 50%]', '[> 100%]']
data = df_operators_participation
data['participation_category'] = pd.cut(data['participation_percentage'], 
                                        bins=range(0, 105, 5),
                                        labels=[f'{i}% - {i+5}%' for i in range(0, 100, 5)]).fillna('0% - 5%')
for seg, df_op in {key: value for key, value in operators_seg_dict.items() if key in segments_list}.items():
    data_aux = data[data.operator_id.isin(df_op.id.unique())]
    data_aux = data_aux.groupby(['participation_category'])['order'].size().reset_index(name='count').sort_values(by='participation_category', ascending=True)
    plt.bar(data_aux['participation_category'], data_aux['count'], label=str(seg))

plt.xlabel('Porcentaje de participación sobre tiempo total OF')
plt.ylabel('Participación operarios')
plt.title('Participación de operarios por línea por segmento de performance')

plt.legend(title='Segmento')
plt.xticks(rotation=90, fontsize=8, ha='center')
plt.show()

In [ ]:
data = df_operators_participation
data['performance_category'] = pd.cut(data['of_performance']*100, 
                                        bins=range(0, 120, 10),
                                        labels=[f'{i}% - {i+10}%' for i in range(0, 110, 10)]).fillna('0% - 10%')

In [ ]:
df_operators_participation.head()

In [ ]:
segments_list= ['[80% - 90%]', '[70% - 80%]', '[60% - 70%]', '[50% - 60%]']
data = df_operators_participation
data['participation_category'] = pd.cut(data['participation_percentage'], 
                                        bins=range(0, 105, 5),
                                        labels=[f'{i}% - {i+5}%' for i in range(0, 100, 5)]).fillna('0% - 5%')
for seg, df_op in {key: value for key, value in operators_seg_dict.items() if key in segments_list}.items():
    data_aux = data[data.operator_id.isin(df_op.id.unique())]
    data_aux = data_aux.groupby(['participation_category'])['order'].size().reset_index(name='count').sort_values(by='participation_category', ascending=True)
    plt.bar(data_aux['participation_category'], data_aux['count'], label=str(seg))

plt.xlabel('Porcentaje de participación sobre tiempo total OF')
plt.ylabel('Participación sobre\ntiempo total de operarios')
plt.title('Participación de operarios por línea por segmento de performance')

plt.legend(title='Segmento')
plt.xticks(rotation=90, fontsize=8, ha='center')
plt.show()

In [ ]:
plt.figure(figsize=(10, 10))
sns.boxplot(data=df_operators_participation, x='performance_category', y='participation_percentage')
plt.xlabel('Performance por participación')
plt.ylabel('Participación Tiempo de producción OF (%)')
plt.title('Porcentaje de Participación en tiempo total \nde producción según segmentos de performance')
plt.xticks(rotation=70, ha='right', fontsize=8)
plt.show()

#### Cantidad de operadores por OF según segmento de performance

In [ ]:
df_of['performance_category'] = pd.cut(df_of['performance']*100, 
                                        bins=range(0, 120, 10),
                                        labels=[f'{i}% - {i+10}%' for i in range(0, 110, 10)]).fillna('0% - 10%')
df_of.head()

In [ ]:
plt.figure(figsize=(10, 10))
sns.boxplot(data=df_of, x='performance_category', y='operators_distinct_qty')
plt.xlabel('Performance')
plt.ylabel('Cantidad de operadores')
plt.title('Cantidad de operadores participantes en OFs según segmentos de performance')
plt.xticks(rotation=70, ha='right', fontsize=12)
plt.show()

In [ ]:
plt.figure(figsize=(10, 10))
sns.boxplot(data=df_of, x='line', y='operators_distinct_qty')
plt.xlabel('Línea')
plt.ylabel('Cantidad de operadores')
plt.title('Cantidad de operadroes participantes en OFs según Línea')
plt.xticks(rotation=70, ha='right', fontsize=12)
plt.show()

In [ ]:
plt.figure(figsize=(10, 10))
sns.boxplot(data=df_of[df_of.registers_qty < 600], x='performance_category', y='registers_qty')
plt.xlabel('Performance')
plt.ylabel('Cantidad de registros')
plt.title('Cantidad de registros en OFs según Performance')
plt.xticks(rotation=70, ha='right', fontsize=12)
plt.show()

In [ ]:
FEATURES = ['good_qty', 'bad_qty', 'time_diff_seconds_calculated', 'time_diff_minutes_calculated',
            'total_operators_minutes', 'registers_qty', 'operators_distinct_qty', 'theorical_qty',
            'performance', 'line', 'bomb_type'
]

In [ ]:
df_of.head()

In [ ]:
sns.pairplot(df_of[FEATURES])

In [ ]:

corrmat = df_of[FEATURES].corr()
hm = sns.heatmap(corrmat, 
                 cbar=True, 
                 annot=True, 
                 square=True, 
                 fmt='.2f', 
                 annot_kws={'size': 10}, 
                 yticklabels=df_of[FEATURES].columns,
                 xticklabels=df_of[FEATURES].columns, 
                 cmap="Spectral_r")
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)
plt.show()

## Guardar tablas como csv

In [ ]:
df_operators_participation.head()

In [ ]:
save_tables = False
if save_tables:
    print(df_operators_participation.head())
    df_operators_participation.to_csv('data/processed_csv/df_operators_participation.csv', index=False)
    print(df_of.head())
    df_of.to_csv('data/processed_csv/df_of.csv', index=False)
    print(df_bombs.head())
    df_bombs.to_csv('data/processed_csv/df_bombs.csv', index=False)
    print(df_operators.head())
    df_operators.to_csv('data/processed_csv/df_operators.csv', index=False)
